In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.utils import to_categorical




In [3]:
english_sentences = ["Hello", "How are you?", "Translate this sentence"]
hindi_sentences = ["नमस्ते", "तुम कैसे हो?", "इस वाक्य का अनुवाद करें"]


In [4]:
english_chars = sorted(list(set(' '.join(english_sentences))))
hindi_chars = sorted(list(set(' '.join(hindi_sentences))))

english_tokenizer = {char: i + 1 for i, char in enumerate(english_chars)}
hindi_tokenizer = {char: i + 1 for i, char in enumerate(hindi_chars)}

num_encoder_tokens = len(english_tokenizer) + 1
num_decoder_tokens = len(hindi_tokenizer) + 1

max_english_sequence_length = max(len(sentence) for sentence in english_sentences)
max_hindi_sequence_length = max(len(sentence) for sentence in hindi_sentences)


In [5]:
def sentences_to_sequences(sentences, tokenizer, max_sequence_length):
    sequences = np.zeros((len(sentences), max_sequence_length))
    for i, sentence in enumerate(sentences):
        for j, char in enumerate(sentence):
            sequences[i, j] = tokenizer[char]
    return sequences

X_train = sentences_to_sequences(english_sentences, english_tokenizer, max_english_sequence_length)
y_train = sentences_to_sequences(hindi_sentences, hindi_tokenizer, max_hindi_sequence_length)

In [6]:
X_train = X_train.reshape((len(english_sentences), max_english_sequence_length, 1))
decoder_input = y_train.reshape((len(hindi_sentences), max_hindi_sequence_length, 1))

In [7]:
y_train_one_hot = to_categorical(y_train.astype(int), num_classes=num_decoder_tokens)

In [8]:
latent_dim = 256

In [9]:
encoder_inputs = Input(shape=(max_english_sequence_length, 1))
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [10]:
decoder_inputs = Input(shape=(max_hindi_sequence_length, 1))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [11]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit([X_train, decoder_input], y_train_one_hot, epochs=10, batch_size=2)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - accuracy: 0.0000e+00 - loss: 3.1672
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.4106 - loss: 2.7291
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.4106 - loss: 2.3736 
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.4928 - loss: 1.9001
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.4348 - loss: 2.0300
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.5531 - loss: 1.5560
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.4517 - loss: 1.7815 
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.5217 - loss: 1.6996
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.5676 - loss: 1.5368
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.6425 - loss: 1.2970


In [14]:
model.save('translation_model.h5')